In [26]:
import numpy as np

def get_combined_X(X_y, RkNN):
    combined = {}
    for i in X_y.keys():
        rknn_lists = [RkNN[x] for x in X_y[i]]
        rknn_lists = [np.array(lst) for lst in rknn_lists]
        combined[i] = np.concatenate(rknn_lists, axis = 0)
        
    return combined

def get_m_tilde(RkNN_X_y):
    m_tilde = {}
    for i in RkNN_X_y.keys():
        m_tilde[i] = np.mean([X[j] for j in RkNN_X_y[i]], axis=0)
    return m_tilde
    
    
if __name__ == "__main__":
    x_1 = np.array([1, 2])
    x_2 = np.array([3, 4])
    x_3 = np.array([10, 12])
    
    RkNN = {
        0: [1],  # x_1의 역 k-최근접 이웃은 x_2입니다.
        1: [0, 2],  # x_2의 역 k-최근접 이웃은 x_1과 x_3입니다.
        2: [0]   # x_3의 역 k-최근접 이웃은 x_1입니다.
    }
    
    # 각 데이터 포인트입니다.
    X = [x_1, x_2, x_3]
    
    X_y = {
        0 : [0, 1], # x1, x2
        1 : [2],
        2 : [0, 1] # x3
    }
    
    RkNN_X_y = get_combined_X(X_y, RkNN)

    print(RkNN_X_y)
        
    m_tilde = get_m_tilde(RkNN_X_y)

    # print(m_tilde)
    
    S_w_nLDA = 0
    
    t = 0
    n = 3
    
    for i in range(n):
        if len(RkNN_X_y[i]) >= t:
            print()
            for j in RkNN_X_y[i]:
                
                x_j = X[j]
                # print(x_j)
                diff = x_j - m_tilde[i]
                # print(diff)
                diff2 = np.dot(diff.T, diff)
                S_w_nLDA += diff2
    
    print(S_w_nLDA)


{0: array([1, 0, 2]), 1: array([0]), 2: array([1, 0, 2])}



201.33333333333337


In [10]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

def get_RkNN(X):
    kNN = get_kNN(X)
    
    RkNN = {i: set() for i in range(len(X))}
    
    for i in kNN.keys():
        for val in kNN[i]:
            RkNN[val].add(i)
        
    
    return RkNN

def get_kNN(X):
    k = 2
    nbrs = NearestNeighbors(n_neighbors=k+1)  # 
    nbrs.fit(X)
    
    _, indices = nbrs.kneighbors(X)

    # print(indices)
    kNN = {i: set() for i in range(len(X))}
    
    for i, neighbors in enumerate(indices):
        for neighbor in neighbors:
            if neighbor != i:
                kNN[i].add(neighbor)
                
    return kNN
    

def get_RkNN_X_y(X_y, RkNN):
    RkNN_X_y = {}
    for i in X_y.keys():
        tmp = np.array([])
        # print()
        for key in X_y[i]:
            rknn_array = np.array(list(RkNN[key]))
            # print(rknn_array)
            tmp = np.concatenate((tmp, rknn_array), axis=0) if tmp.size else rknn_array
        RkNN_X_y[i] = tmp
    return RkNN_X_y

def get_app_RkNN_X_y(app_X_y, RkNN):
    RkNN_X_y = {}
    for i in app_X_y.keys():
        tmp = np.array([])
        # print()
        for key in app_X_y[i]:
            rknn_array = np.array(list(RkNN[key]))
            # print(rknn_array)
            # print(rknn_array)
            tmp = np.concatenate((tmp, rknn_array), axis=0) if tmp.size else rknn_array
        RkNN_X_y[i] = tmp
    return RkNN_X_y



def get_m_tilde(RkNN_X_y, X):
    m_tilde = {}
    for i in RkNN_X_y.keys():
        if len(RkNN_X_y[i]) != 0:
            m_tilde[i] = np.mean([X[j] for j in RkNN_X_y[i]], axis=0)
        else:
            m_tilde[i] = np.array([])
    
    return m_tilde

def get_S_w_nLDA(m_tilde, RkNN_X_y, X):
    S_w_nLDA = 0
    
    t = 0
    n = 4
    
    for i in range(n):
        if len(RkNN_X_y[i]) >= t:
            # print()
            for j in RkNN_X_y[i]:
                
                x_j = X[j]
                # print(x_j)
                diff = (x_j - m_tilde[i]).reshape(1, 2)
                
                diff2 = np.dot(diff.T, diff)
                S_w_nLDA += diff2
                
    return S_w_nLDA

def get_S_b_nLDA(m_tilde, RkNN_X_y):
    S_b_nLDA = 0
    cnt = 0
    
    t = 0
    n = 4
    
    for i in range(n):
        if len(RkNN_X_y[i]) >= t:
            # print()
            for j in range(n):
                # print(j)
                if( not(np.array_equal(RkNN_X_y[i], RkNN_X_y[j])) and (len(RkNN_X_y[j]) >= t)):
                    # print(j)
                    diff = (m_tilde[i] - m_tilde[j]).reshape(1, 2)
                    
                    diff2 = np.dot(diff.T, diff)
                    cnt += 1
                    
                    S_b_nLDA += diff2
    print(cnt)
    return S_b_nLDA


def get_app_X_y(X_y, X):
    kNN = get_kNN(X)
    app_X_y = {i: set() for i in range(len(X))}

    for i in kNN.keys():
        # print()
        for x in kNN[i]:
            # print(x)
            # print(X_y[i])
            if x not in X_y[i]:
                # print(x)
                app_X_y[i].add(x)
    return app_X_y

x_1 = np.array([1, 2])
x_2 = np.array([3, 4])
x_3 = np.array([10, 12])
x_4 = np.array([8, 9])

X = np.array([x_1, x_2, x_3, x_4])
Y = np.array([0, 1, 1, 1])

X_y = {
    0 : [0],
    1 : [1, 2, 3],
    2 : [1, 2, 3],
    3 : [1, 2, 3]
}

kNN = get_kNN(X)
# print(kNN)

RkNN = get_RkNN(X)
# print(RkNN)

RkNN_X_y = get_RkNN_X_y(X_y, RkNN)
# print(RkNN_X_y)


m_tilde = get_m_tilde(RkNN_X_y, X)
print(m_tilde)


S_w_nLDA = get_S_w_nLDA(m_tilde, RkNN_X_y, X)
print(S_w_nLDA)


S_b_nLDA = get_S_b_nLDA(m_tilde, RkNN_X_y)
print(S_b_nLDA)




{0: array([3., 4.]), 1: array([5.85714286, 7.14285714]), 2: array([5.85714286, 7.14285714]), 3: array([5.85714286, 7.14285714])}
[[296.57142857 321.42857143]
 [321.42857143 350.57142857]]
6
[[48.97959184 53.87755102]
 [53.87755102 59.26530612]]


In [20]:

def get_app_S_b_nLDA(app_m_tilde, m_tilde, app_X_y, X_y):
    S_b_nLDA = 0
    
    t = 1
    n = 4
    cnt = 0

    for i in range(n):
        if len(RkNN_X_y[i]) >= t:
            print()
            for j in range(n):
                # print(RkNN_X_y[i])
                # print(app_RkNN_X_y[j])
                
                if( not(np.array_equal(RkNN_X_y[i], RkNN_X_y[j])) and (len(app_RkNN_X_y[j]) >= t)):
                    
                    diff = (m_tilde[i] - app_m_tilde[j]).reshape(1, 2)
                    diff2 = np.dot(diff.T, diff)
                    S_b_nLDA += diff2
                    cnt += 1
    # print(cnt)
    return S_b_nLDA

RkNN = get_RkNN(X)

app_X_y = get_app_X_y(X_y, X)
# print(app_X_y)

app_RkNN_X_y = get_app_RkNN_X_y(app_X_y, RkNN)
# print(app_RkNN_X_y)

app_m_tilde = get_m_tilde(app_RkNN_X_y, X)
# print(app_m_tilde)

RkNN = get_RkNN(X)
RkNN_X_y = get_RkNN_X_y(X_y, RkNN)
# print(RkNN_X_y)

app_S_b_nLDA = get_app_S_b_nLDA(app_m_tilde, m_tilde, app_X_y, X_y)

print(app_S_b_nLDA)






[[0.38265306 0.33163265]
 [0.33163265 0.28741497]]


In [24]:
import numpy as np
from scipy.linalg import eigh


eigenvalues, eigenvectors = eigh(app_S_b_nLDA, S_w_nLDA)

print("고유값:", eigenvalues)
print("고유벡터:\n", eigenvectors)


고유값: [2.60208521e-18 9.48318960e-03]
고유벡터:
 [[ 0.21541373 -0.70024448]
 [-0.2485543   0.62632978]]
